In [ ]:
# Python modules
import flickrapi
import pandas as pd
import os
import json
from dotenv import load_dotenv
load_dotenv()
import numpy as np

# Import additional functions
from flickr_functions import df_remove_dupes

# Keys needed for API access
api_key = os.getenv('flickr_api_key')
api_secret = os.getenv('flickr_api_secret')

# Flickr API object
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

# Data directory used to store CVS files
data_dir = './data/'

In [ ]:
# Load EXIF dataframe from CVS file
df_exif = pd.read_csv(data_dir + 'df_photo_exif_final.csv', index_col=[0])
#df_tags = pd.DataFrame(columns=['id', 'tags'])
df_tags = pd.read_csv(data_dir + 'df_photo_tags.csv', index_col=[0])

In [ ]:
# Go through ids from exif dataframe that are not in tags dataframe yet
for index, row in df_exif[~df_exif.id.isin(df_tags.id)].iterrows():

    # Catch exceptions so the script can live on
    try:
        # Asking the API for the list of tags of a image id
        tags = flickr.tags.getListPhoto(photo_id = int(row.get('id'))).get('photo').get('tags').get('tag')
        # Add the tags to a temporary list
        tmp_list = []
        for tag in tags:
            tmp_list.append(tag.get('raw'))
        # Add the tmp list to the bottom of the tags dataframe
        df_tags.loc[len(df_tags)] = {'id': row.get('id'), 'tags': tmp_list}
        # Print some info
        print(f'{int(row.get("id"))}: {tmp_list}')
    except flickrapi.exceptions.FlickrError as ex:
        # If accessing the API fails, print something out
        print(f'!!! Error code: {ex.code} for id {int(row["id"])}')

# Write to csv
df_tags.to_csv(data_dir + 'df_photo_tags.csv')

In [ ]:
df_tags.to_csv(data_dir + 'df_photo_tags.csv')